**Работа с сетями в docker**

Реализуется через API network.

# `ls`

Выведет список доступных сетей.

In [1]:
%%bash
docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
cac8977da21a   bridge    bridge    local
f8b2503d0640   host      host      local
b1d7e6bda275   none      null      local


# Подключение к сети

Производится, через указывание опции `--net=<имя/id сети>`. Далее в примере демонстрирую, что как минимум ничего не сломалось.

In [5]:
%%bash
docker run --rm -di --net=none --name just_net ubuntu_curl
docker stop just_net

ee6e4f29315a7fcf3a5ad2ee39cdb86bb178ebb296563fee73b432a30799a33c
just_net


# Сети по умолчанию

В docker по умолчанию доступно 3 сети:

In [2]:
%%bash
docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
cac8977da21a   bridge    bridge    local
f8b2503d0640   host      host      local
b1d7e6bda275   none      null      local


### Сеть none

Производит полное блокирование контейнера от сети.

Так попробуем запустить clickhouse на этой сети и сделать к нему запрос. `curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused` свидетельствует о том, что по заданному адресу ничего не отвечат.

In [5]:
%%bash
docker run --rm -d --name none_example -p 8123:8123 --net=none yandex/clickhouse-server
# нельзя забывать, что clickhouse долговато поднимается
sleep 10
curl localhost:8123
docker stop none_example

05db6909cc4de9e808ffe927ad06594fafce9ea6e0a54e6fadf38f04e266759b


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused


none_example


Так же из нутри контейнера, нельзя будет достучаться в интернет. Так в следующем примере я запускаю ubuntu с предустановленным curl и получаю, и не могу из него достучаться до google.com. `curl: (6) Could not resolve host: google.com` свидетельсвует о том, что из контейнера не получается достучаться до google.com.

In [2]:
%%bash
docker run -itd --rm --net=none --name none_example ubuntu_curl
docker exec none_example curl google.com
docker stop none_example

3dd6e8e5f072ff86380acf7f092093bb96a1a1d7a3b091aaaf32d6ee21763e3c


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: google.com


none_example


### Сеть host

Полностью убирает сетевую изоляцию контейнера, то есть контейнер делит с хостом сеть полностью.

Так в примере я запускаю yandex/clickhouse-server без указания каких-либо портов. В результате, этот clickhouse всеравно будет отвечать (даже без указанных портов).

In [6]:
%%bash
docker run --rm -d --name host_example --net=host yandex/clickhouse-server
# нельзя забывать, что clickhouse долговато поднимается
sleep 10
curl localhost:8123
docker stop host_example

69120ad84831e74d88b6085e5b38ca8ba2a0f05ff9b66ac2748a01b7974707e6


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   4149      0 --:--:-- --:--:-- --:--:--  4000


Ok.
host_example


### Сеть bridge

Сеть в которой контейнеры создаются по умолчанию. Далее показано несколько трюков.

**Весть этот раздел воспроиводится особым образом**

Я предварительно поднял контейнеры с yandex/clickhouse-server и ubuntu с предустановленным curl:

In [7]:
%%bash
docker ps

CONTAINER ID   IMAGE                      COMMAND            CREATED              STATUS              PORTS                          NAMES
3c653442fea3   ubuntu_curl                "/bin/bash"        About a minute ago   Up About a minute                                  ubu
3f047e11c417   yandex/clickhouse-server   "/entrypoint.sh"   2 minutes ago        Up 2 minutes        8123/tcp, 9000/tcp, 9009/tcp   click


Теперь демонстрирую результат комманды `insperct` (позволяет получить низкоуровневую инофрмацию о любом объекте docker) для сети bridge.

In [8]:
%%bash
docker inspect bridge

[
    {
        "Name": "bridge",
        "Id": "243b737447851abe893a50fab43e6fd6304423b81f848e31ae974692bb2388a5",
        "Created": "2023-03-22T16:23:03.478736367+03:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "3c653442fea36887a8ff860b337148053ba5583c74ad635eeaa0bce4f75c8432": {
                "Name": "ubu",
                "EndpointID": "f29acdda0cc093af06a9797386eed25d491bb67e9e09112bae235657c81f93c9",
                "MacAddress": "02:42:ac:11:00:03",
                "IPv4Address"

Обращаю внимание на поле "Containers" а в нем на поля "IPv4Address". По этим адресам мы можем общаться между контейнерами. Так вот например и из ubuntu спрошу clikchouse. Ok свидетельсвует о том, что я дейвительно дотянулся.

In [9]:
%%bash
docker exec ubu curl 172.17.0.2:8123

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   4944      0 --:--:-- --:--:-- --:--:--  4000


Ok.


Более того, несмотря на то, что я не указывал портов при поднятии контейнера clickhouse, через этот ip я могу достучаться до clickhouse и из хоста. В следующем примере, я сравниваю результат запроса на localhost и на этот ip.

- В первом случае никуда я не достучался и закономерно получил `curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused`;
- Во втором случа clickhouse мне отвечает как ни в чем не бывало `Ok.`.

In [10]:
%%bash
curl localhost:8123
curl 172.17.0.2:8123

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   9302      0 --:--:-- --:--:-- --:--:--  4000


Ok.
